## Importing Libraries

In [1]:
import io

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)
# Make sure you are running this in Python3 environment and TFv2.x

2.2.0


## Loading Data

In [2]:
dataset, info = tfds.load(
    'imdb_reviews/subwords8k', 
    with_info=True, 
    as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Shuffling and writing examples to C:\Users\Parivi\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompletePS25ST\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\Parivi\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompletePS25ST\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\Parivi\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0.incompletePS25ST\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\Parivi\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0. Subsequent calls will reuse this data.


## Tokenizing and Padding

In [3]:
tokenizer = info.features['text'].encoder

BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(
    BATCH_SIZE, 
    tf.compat.v1.data.get_output_shapes(train_dataset)
)
test_dataset = test_dataset.padded_batch(
    BATCH_SIZE, 
    tf.compat.v1.data.get_output_shapes(test_dataset)
)

## Building a Model

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


## Fitting

In [ ]:
NUM_EPOCHS = 10
history = model.fit(
    train_dataset, 
    epochs=NUM_EPOCHS,
    validation_data=test_dataset
)

## Visualising Accuracy and Loss

In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history, 'accuracy')

plot_graphs(history, 'loss')